In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# GCP上的端到端机器学习：MLOps阶段2：实验：开始使用Vertex AI特征存储

<table align="left">
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage2/get_started_vertex_feature_store.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
    
  <td>
        <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage2/get_started_vertex_feature_store.ipynb">
        <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
        </a>
  </td>
    
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/community/ml_ops/stage2/get_started_vertex_feature_store.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在Vertex AI工作台中打开
    </a>
  </td>
    
</table>
<br/><br/><br/>

## 概览

本教程演示了如何在 Google 云上将 Vertex AI 用于端到端 MLOps 生产环境。本教程涵盖了第二阶段：实验：开始使用特征存储。

### 目标

在本教程中，您将学习如何在训练和预测时使用 `Vertex AI Feature Store`。

此教程使用以下谷歌云ML服务：

- `Vertex AI Feature Store`

执行的步骤包括：

- 创建Vertex AI `Featurestore` 资源。
    - 为 `Featurestore` 资源创建 `EntityType` 资源。
    - 为每个 `EntityType` 资源创建 `Feature` 资源。
- 将特征值（实体数据项）导入到 `Featurestore` 资源中。
    - 从Cloud Storage位置。
    - 从pandas DataFrame。
- 从 `Featurestore` 资源执行在线服务。
- 从 `Featurestore` 资源执行批处理服务。

### 数据集

本教程使用的数据集是“电影推荐”数据集。在教程中使用的数据集版本存储在一个公共云存储桶中，格式为Avro。

该数据集用于预测一个人是否观看电影。

### 费用
此教程使用 Google Cloud 的计费组件：

- Vertex AI
- 云存储
- BigQuery

了解 [Vertex AI 价格](https://cloud.google.com/vertex-ai/pricing)、[云存储价格](https://cloud.google.com/storage/pricing) 和 [BigQuery 价格](https://cloud.google.com/bigquery/pricing)，并使用 [定价计算器](https://cloud.google.com/products/calculator/) 根据您的预期使用情况生成费用估算。

安装以下软件包以进一步运行此笔记本。

In [ ]:
import os

# The Vertex AI Workbench Notebook product has specific requirements
IS_WORKBENCH_NOTEBOOK = os.getenv("DL_ANACONDA_HOME") and not os.getenv("VIRTUAL_ENV")
IS_USER_MANAGED_WORKBENCH_NOTEBOOK = os.path.exists(
    "/opt/deeplearning/metadata/env_version"
)

# Vertex AI Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_WORKBENCH_NOTEBOOK:
    USER_FLAG = "--user"

# Install the dependecies
! pip3 install --upgrade google-cloud-aiplatform google-cloud-bigquery pyarrow avro $USER_FLAG -q

### 重新启动内核

安装了额外的包之后，您需要重新启动笔记本内核，以便它可以找到这些包。

In [ ]:
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## 开始之前

### 设置你的 Google Cloud 项目

**无论你使用什么笔记本环境，都需要按照以下步骤进行操作。**

1. [选择或创建一个 Google Cloud 项目](https://console.cloud.google.com/cloud-resource-manager)。当你首次创建账号时，你将获得$300的免费信用额度，可用于支付计算/存储成本。

1. [确保你的项目已启用计费功能](https://cloud.google.com/billing/docs/how-to/modify-project)。

1. [启用 Vertex AI、Compute Engine、Cloud Storage 和 Cloud Logging API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,compute_component,storage_component,logging)。

1. 如果你在本地运行这个笔记本，你需要安装 [Cloud SDK](https://cloud.google.com/sdk)。

1. 在下面的单元格中输入你的项目 ID。然后运行这个单元格，确保 Cloud SDK 在本笔记本中的所有命令中使用正确的项目。

**注意**：Jupyter 运行以 `!` 开头的行作为 shell 命令，并将以 `$` 开头的 Python 变量插入这些命令中。

#### 设置您的项目ID

**如果您不知道您的项目ID**，您可以尝试使用`gcloud`来获取您的项目ID。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

#### 区域

您还可以更改“REGION”变量，该变量用于笔记本的其余部分操作。以下是Vertex AI支持的区域。我们建议您选择离您最近的区域。

- 美洲: `us-central1`
- 欧洲: `europe-west4`
- 亚太: `asia-east1`

您可能无法使用多区域存储桶进行Vertex AI培训。并非所有区域都支持所有Vertex AI服务。

了解更多关于[Vertex AI区域](https://cloud.google.com/vertex-ai/docs/general/locations)。

In [ ]:
REGION = "[your-region]"  # @param {type: "string"}

if REGION == "[your-region]":
    REGION = "us-central1"

时间戳

如果您在进行实时教程会话，您可能正在使用共享的测试账户或项目。为了避免用户在创建资源时发生名称冲突，您可以为每个实例会话创建一个时间戳，并将该时间戳附加到您在本教程中创建的资源名称上。

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### 认证您的Google Cloud帐户

**如果您正在使用Vertex AI Workbench笔记本**，则您的环境已经经过验证。跳过这一步。

**如果您正在使用Colab**，运行下面的单元格，并按照提示进行身份验证，通过oAuth认证您的帐户。

**否则**，按照以下步骤操作：

在Cloud Console中，转到[创建服务帐号密钥](https://console.cloud.google.com/apis/credentials/serviceaccountkey)页面。

1. **点击创建服务帐号**。

2. 在**服务帐号名称**字段中输入一个名称，然后点击**创建**。

3. 在**将此服务帐号授予项目访问权限**部分，点击角色下拉列表。在过滤框中输入"Vertex"，选择**Vertex管理员**。在过滤框中输入"Storage Object Admin"，选择**Storage Object Admin**。

4. 点击创建。下载包含您密钥的JSON文件到您的本地环境。

5. 在下面的单元格中输入您的服务帐号密钥路径作为GOOGLE_APPLICATION_CREDENTIALS变量，然后运行该单元格。

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

import os
import sys

# If on Vertex AI Workbench, then don't execute this code
IS_COLAB = "google.colab" in sys.modules
if not os.path.exists("/opt/deeplearning/metadata/env_version") and not os.getenv(
    "DL_ANACONDA_HOME"
):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

导入库并定义常量

In [ ]:
import google.cloud.aiplatform as aiplatform
from google.cloud import bigquery

初始化Vertex AI和BigQuery客户端。

In [ ]:
aiplatform.init(project=PROJECT_ID)
bqclient = bigquery.Client(project=PROJECT_ID)

## 顶点 AI 特征存储介绍

假设您有一个推荐模型，该模型可预测在收银机小票背面打印的优惠券。 现在，如果该模型仅在单个交易实例上进行了训练（购买了什么以及金额），那么（过去）您曾使用 Apriori 算法。

但现在我们有客户的历史数据（例如按信用卡号编制索引）。 比如迄今为止的总购买金额、每笔交易平均购买金额、按产品类别购买频率等。 我们使用这些“丰富的数据”来训练推荐系统。

现在到了进行实时预测的时候。 您从收银机获得一笔交易，但其中只有信用卡号和此交易。 它没有模型需要的丰富数据。 在提供服务时，信用卡号被用作 Feature Store 的索引，以获取模型所需的丰富数据。

另一方面，假设模型训练所用的丰富数据是在 6 月 1 日时间戳上的。 当前交易是从 6 月 15 日。 假设用户在 6 月 1 日至 15 日之间进行了其他交易，并且 Feature Store 中的丰富数据已经不断更新。 但是模型是基于 6 月 1 日的数据进行训练的。 Feature Store 知道版本号，并向模型提供 6 月 1 日版本（而不是当前的 6 月 15 日）。 否则，如果使用 6 月 15 日的数据，将会出现训练与提供的偏差。

这里另一个问题是数据漂移。 事物会发生变化，突然有一天，每个人都在购买卫生纸！ 现有丰富数据的分布发生了显着变化，与部署模型训练时的分布不同。 Feature Store 可以检测到分布变化/阈值变化，并触发重新训练模型的通知。

了解有关 [顶点 AI 特征存储 API](https://cloud.google.com/vertex-ai/docs/featurestore) 的更多信息。

## Vertex AI 特征存储数据模型

Vertex AI 特征存储使用以下三个重要的层次概念组织数据：

        特征存储 -> 实体类型 -> 特征

- `特征存储`：存储特征的地方。
- `实体类型`：在一个`特征存储`下，`实体类型`描述了要建模的对象，可以是真实的也可以是虚拟的。
- `特征`：在一个`实体类型`下，`特征`描述了`实体类型`的属性。

了解更多关于[Vertex AI 特征存储数据模型](https://cloud.google.com/vertex-ai/docs/featurestore/concepts)。

在电影预测数据集中，您创建了一个名为movies的`特征存储`资源。这个`特征存储`资源有两个实体类型：
- `users`：这个实体类型具有`年龄`、`性别`和`喜欢的流派`属性。
- `movies`：这个实体类型具有`流派`和`平均评分`属性。

## 创建`Featurestore`资源

首先，您可以使用`Featurestore.create()`方法为数据集创建一个`Featurestore`，需要提供以下参数：

- `featurestore_id`：特征存储的名称。
- `online_store_fixed_node_count`：用于在线服务特征存储的配置设置。
- `project`：项目ID。
- `location`：地点（地区）。

In [ ]:
# Represents featurestore resource path.
FEATURESTORE_NAME = "movies_" + TIMESTAMP

featurestore = aiplatform.Featurestore.create(
    featurestore_id=FEATURESTORE_NAME,
    online_store_fixed_node_count=1,
    project=PROJECT_ID,
    location=REGION,
)

print(featurestore)

### 列出您的“Featurestore”资源

您可以使用`Featurestore.list()`方法获取项目中所有`Featurestore`资源的列表。

In [ ]:
for featurestore in aiplatform.Featurestore.list():
    print(featurestore)

获取`Featurestore`资源

您可以使用`Featurestore（）`初始化程序获取项目中指定的`Featurestore`资源，具有以下参数：

- `featurestore_name`：`Featurestore`资源的名称。
- `project`：项目ID。
- `location`：位置（地区）。

In [ ]:
featurestore = featurestore = aiplatform.Featurestore(
    featurestore_name=FEATURESTORE_NAME, project=PROJECT_ID, location=REGION
)
print(featurestore)

为您的“Featurestore”资源创建实体类型

接下来，您可以使用“create_entity_type()”方法为您的“Featurestore”资源创建“EntityType”资源，使用以下参数：

- “entity_type_id”：实体类型资源的名称。
- “description”：实体类型的描述。

In [ ]:
for name, description in [("users", "Users descrip"), ("movies", "Movies descrip")]:
    entity_type = featurestore.create_entity_type(
        entity_type_id=name, description=description
    )
    print(entity_type)

### 为您的`EntityType`资源添加`Feature`资源

接下来，您可以使用`create_feature()`方法为您的`Featurestore`资源中的每个`EntityType`资源创建`Feature`资源，使用以下参数：

- `feature_id`：`Feature`资源的名称。
- `description`：对该特征的描述。
- `value_type`：该特征的数据类型。

In [ ]:
def create_features(featurestore_name, entity_name, features):
    entity_type = aiplatform.EntityType(
        entity_type_name=entity_name, featurestore_id=featurestore_name
    )

    for feature in features:
        feature = entity_type.create_feature(
            feature_id=feature[0], description=feature[1], value_type=feature[2]
        )
        print(feature)


create_features(
    FEATURESTORE_NAME,
    "users",
    [
        ("age", "Age descrip", "INT64"),
        ("gender", "Gender descrip", "STRING"),
        ("liked_genres", "Genres descrip", "STRING_ARRAY"),
    ],
)

create_features(
    FEATURESTORE_NAME,
    "movies",
    [
        ("title", "Title descrip", "STRING"),
        ("genres", "Genres descrip", "STRING"),
        ("average_rating", "Ave descrip", "DOUBLE"),
    ],
)

### 列出您的 `Featurestore` 资源

您可以使用 `Featurestore.list()` 方法获取您项目中所有 `Featurestore` 资源的列表。

In [ ]:
for featurestore in aiplatform.Featurestore.list():
    print(featurestore)

### 使用过滤器搜索 `特征` 资源

您可以通过使用 `list_features()` 方法并指定一个 `filter` 字符串来缩小对 `特征` 资源的搜索范围。

In [ ]:
# Search by data type
features = []
for entity_type in featurestore.list_entity_types():
    features += entity_type.list_features(filter="value_type=DOUBLE")
print("By data type")
for feature in features:
    print(features)

# Search by data type
features = []
for entity_type in featurestore.list_entity_types():
    _ = entity_type.list_features()
    for feature in _:
        if feature.name == "title":
            print(type(feature))
            features += [feature]

print("By Name")
for feature in features:
    print(features)

### 使用查询搜索“功能”资源

您可以使用`search()`方法并指定`query`过滤器来缩小对“功能”资源的搜索范围。

In [ ]:
features = aiplatform.Feature.search(query="value_type=DOUBLE")
print("By data type")
for feature in features:
    print(features)

aiplatform.Feature.search(query="feature_id=title")
print("By Name")
for feature in features:
    print(features)

定义通往特征数据的路径。

In [ ]:
IMPORT_FILE = (
    "gs://cloud-samples-data/vertex-ai/feature-store/datasets/movie_prediction.csv"
)
FS_ENTITIES = {
    "users": "gs://cloud-samples-data/vertex-ai/feature-store/datasets/users.avro",
    "movies": "gs://cloud-samples-data-us-central1/vertex-ai/feature-store/datasets/movies.avro",
}

## 将特征数据导入到您的`Featurestore`资源

接下来，您将导入您的`Featurestore`资源的特征数据。一旦导入，您可以在在线和离线（批量）服务中使用这些特征值。

### 数据布局

每个导入的`EntityType`资源数据必须具有ID。此外，每个`EntityType`资源数据项可以选择性地具有一个时间戳，指定生成特征值的时间。

在导入时，在您的请求中指定以下内容：

- 数据源格式：BigQuery表/Avro/CSV/Pandas Dataframe
- 数据源URL
- 目的地：要导入的特征存储/实体类型/特征

`电影推荐`数据集的特征值以Avro格式存储。Avro模式如下：

**用户实体**：

```
schema = {
  "type": "record",
  "name": "User",
  "fields": [
      {
       "name":"user_id",
       "type":["null","string"]
      },
      {
       "name":"age",
       "type":["null","long"]
      },
      {
       "name":"gender",
       "type":["null","string"]
      },
      {
       "name":"liked_genres",
       "type":{"type":"array","items":"string"}
      },
      {
       "name":"update_time",
       "type":["null",{"type":"long","logicalType":"timestamp-micros"}]
      },
  ]
 }
 ```

**电影实体**：

```
schema = {
 "type": "record",
 "name": "Movie",
 "fields": [
     {
      "name":"movie_id",
      "type":["null","string"]
     },
     {
      "name":"average_rating",
      "type":["null","double"]
     },
     {
      "name":"title",
      "type":["null","string"]
     },
     {
      "name":"genres",
      "type":["null","string"]
     },
     {
      "name":"update_time",
      "type":["null",{"type":"long","logicalType":"timestamp-micros"}]
     },
 ]
}
```

### 从云存储导入特征值

您可以使用`ingest_from_gcs()`方法导入`EntityType`资源的特征值，参数如下：

- `entity_id_field`：父`EntityType`资源的标识名称。
- `feature_ids`：要添加到`EntityType`资源的`Feature`资源数据的标识名称列表。
- `feature_time`：对应于要输入的特征的时间戳字段。
- `gcs_source_type`：导入数据的格式。必须为CSV或Avro。
- `gcs_source_uris`：导入数据文件的一个或多个云存储位置的列表。

In [ ]:
entity_type = featurestore.get_entity_type("users")
response = entity_type.ingest_from_gcs(
    entity_id_field="user_id",
    feature_ids=["age", "gender", "liked_genres"],
    feature_time="update_time",
    gcs_source_type="avro",
    gcs_source_uris=[FS_ENTITIES["users"]],
)
print(response)

entity_type = featurestore.get_entity_type("movies")
response = entity_type.ingest_from_gcs(
    entity_id_field="movie_id",
    feature_ids=["title", "genres", "average_rating"],
    feature_time="update_time",
    gcs_source_type="avro",
    gcs_source_uris=[FS_ENTITIES["movies"]],
)
print(response)

#### 删除实体类型及其对应的特征和特征值

现在，为了准备重复导入特征值的过程，但这次是从数据框中导入，您需要删除现有的实体类型及其对应的内容。

In [ ]:
entity_type = featurestore.get_entity_type("users")
entity_type.delete(force=True)
entity_type = featurestore.get_entity_type("movies")
entity_type.delete(force=True)

## 为您的 `Featurestore` 资源创建实体类型

接下来，您可以再次使用 `create_entity_type()` 方法为您的 `Featurestore` 资源创建 `EntityType` 资源，具体参数如下：

- `entity_type_id`：`EntityType` 资源的名称。
- `description`：实体类型的描述。

In [ ]:
for name, description in [("users", "Users descrip"), ("movies", "Movies descrip")]:
    entity_type = featurestore.create_entity_type(
        entity_type_id=name, description=description
    )
    print(entity_type)

### 为您的`EntityType`资源添加`Feature`资源

此外，您可以使用`create_feature()`方法为您的`Featurestore`资源中的每个`EntityType`资源再次创建`Feature`资源，具有以下参数：

- `feature_id`：`Feature`资源的名称。
- `description`：特征的描述。
- `value_type`：特征的数据类型。

In [ ]:
def create_features(featurestore_name, entity_name, features):
    entity_type = aiplatform.EntityType(
        entity_type_name=entity_name, featurestore_id=featurestore_name
    )

    for feature in features:
        feature = entity_type.create_feature(
            feature_id=feature[0], description=feature[1], value_type=feature[2]
        )
        print(feature)


create_features(
    FEATURESTORE_NAME,
    "users",
    [
        ("age", "Age descrip", "INT64"),
        ("gender", "Gender descrip", "STRING"),
        ("liked_genres", "Genres descrip", "STRING_ARRAY"),
    ],
)

create_features(
    FEATURESTORE_NAME,
    "movies",
    [
        ("title", "Title descrip", "STRING"),
        ("genres", "Genres descrip", "STRING"),
        ("average_rating", "Ave descrip", "DOUBLE"),
    ],
)

现在，将`users`和`movies`数据复制到avro文件中。

In [ ]:
GCS_USERS_AVRO_URI = FS_ENTITIES["users"]
GCS_MOVIES_AVRO_URI = FS_ENTITIES["movies"]

USERS_AVRO_FN = "users.avro"
MOVIES_AVRO_FN = "movies.avro"

! gsutil cp $GCS_USERS_AVRO_URI $USERS_AVRO_FN
! gsutil cp $GCS_MOVIES_AVRO_URI $MOVIES_AVRO_FN

将Avro文件加载到pandas DataFrames中

In [ ]:
from avro.datafile import DataFileReader
from avro.io import DatumReader


class AvroReader:
    def __init__(self, data_file):
        self.avro_reader = DataFileReader(open(data_file, "rb"), DatumReader())

    def to_dataframe(self):
        records = [record for record in self.avro_reader]
        return pd.DataFrame.from_records(data=records)


import pandas as pd

users_avro_reader = AvroReader(data_file=USERS_AVRO_FN)
users_source_df = users_avro_reader.to_dataframe()
print(users_source_df)

movies_avro_reader = AvroReader(data_file=MOVIES_AVRO_FN)
movies_source_df = movies_avro_reader.to_dataframe()
print(movies_source_df)

### 从DataFrame导入特征值

您可以使用 `ingest_from_df()` 方法导入 `EntityType` 资源的特征值，需要使用以下参数：

- `entity_id_field`：父级 `EntityType` 资源的标识符名称。
- `feature_ids`：要添加到 `EntityType` 资源中的 `Feature` 资源数据的标识符名称列表。
- `feature_time`：与正在输入的特征时间戳对应的字段。
- `df_source`：包含导入特征值的DataFrame。

In [ ]:
entity_type = featurestore.get_entity_type("users")
entity_type.ingest_from_df(
    feature_ids=["age", "gender", "liked_genres"],
    feature_time="update_time",
    df_source=users_source_df,
    entity_id_field="user_id",
)

entity_type = featurestore.get_entity_type("movies")
entity_type.ingest_from_df(
    feature_ids=["average_rating", "title", "genres"],
    feature_time="update_time",
    df_source=movies_source_df,
    entity_id_field="movie_id",
)

Vertex AI功能存储服务

Vertex AI功能存储服务为`Featurestore`资源提供以下两项服务：

- 在线服务 - 低延迟提供小批量功能（预测）。

- 批处理服务 - 高吞吐量提供大批量功能（训练和预测）。

In [ ]:
def serve_features(featurestore, entity_name, features, id):
    entity_type = featurestore.get_entity_type(entity_name)
    return entity_type.read(entity_ids=[id], feature_ids=features)


features = serve_features(
    featurestore, "users", ["age", "gender", "liked_genres"], "alice"
)
print(features)

批量服务

Vertex AI Feature Store的批处理服务针对以高吞吐量实时提供大批量特征而进行了优化，通常用于训练模型或批量预测。

可以批量提供到以下目的地：

- BigQuery表
- 云存储位置
- Dataframe

### 输出数据集

对于批量服务，您可以使用BigQuery表作为输出。首先，您必须创建这个输出目标表。

In [ ]:
# Output dataset
DESTINATION_DATASET = f"movies_predictions_{TIMESTAMP}"

# Output table.
DESTINATION_TABLE = "training_data"  # @param {type:"string"}

DESTINATION_TABLE_URI = f"bq://{PROJECT_ID}.{DESTINATION_DATASET}.{DESTINATION_TABLE}"

dataset_id = f"{PROJECT_ID}.{DESTINATION_DATASET}"
dataset = bigquery.Dataset(dataset_id)
dataset = bqclient.create_dataset(dataset)
print("Created dataset:", dataset_id)

批量读取特征数值

您可以使用`read_serve_to_bq()`方法批量将实体数据项提供给BigQuery表，具有以下参数：

- `bq_destination_output_uri`：接收提供特征的目标BigQuery表。
- `serving_feature_ids`：要提供的实体类型和对应的特征字典。
- `read_instances_uri`：从Cloud Storage位置读取实体数据项。

输出存储在一个BigQuery表中。

In [ ]:
response = featurestore.batch_serve_to_bq(
    bq_destination_output_uri=DESTINATION_TABLE_URI,
    serving_feature_ids={
        "users": ["age", "gender", "liked_genres"],
        "movies": ["average_rating", "genres"],
    },
    read_instances_uri=IMPORT_FILE,
)

print(response)

## 清理
### 删除一个BigQuery数据集

使用`delete_dataset()`方法来删除一个BigQuery数据集以及其中所有的表，可以通过将参数`delete_contents`设置为`True`来实现。

In [ ]:
bqclient.delete_dataset(dataset, delete_contents=True)

### 删除`Featurestore`资源

您可以使用`delete()`方法删除指定的`Featurestore`资源，使用以下参数：

- `force`: 一个标识是否删除非空`Featurestore`资源的标志。

In [ ]:
featurestore.delete(force=True)